### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/movies.dat', delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], dtype={'movie_id': object}, engine='python')
reviews = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/ratings.dat', delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, engine='python')

#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [7]:
# number of movies
print("The number of movies is {}.".format(movies.shape[0]))

# number of ratings
print('The number of ratings is {}.'.format(reviews.shape[0]))

# unique users
print("The number of unique users is {}.".format(reviews.user_id.nunique()))

# missing ratings
print('The number of missing reviews is {}.'.format(int(reviews.rating.isnull().mean()*reviews.shape[0])))

# the average, min, and max ratings given
print("The average, minimum, and max ratings given are {}, {}, and {}, respectively.".format(np.round(reviews.rating.mean(), 
                                                                                                      0),
                                                                                             reviews.rating.min(), 
                                                                                             reviews.rating.max()))

The number of movies is 36966.
The number of ratings is 898040.
The number of unique users is 70191.
The number of missing reviews is 0.
The average, minimum, and max ratings given are 7.0, 0, and 10, respectively.


In [9]:
# number of different genres
genres = []

for val in movies.genre:
    try:
        genres.extend(val.split('|'))
    except AttributeError:
        pass
    
genres = set(genres)
print('The number of genres is {}.'.format(len(genres)))

The number of genres is 28.


In [13]:
# Use your findings to match each variable to the correct statement in the dictionary
a = 53968
b = 10
c = 7
d = 31245
e = 15
f = 0
g = 4
h = 712337
i = 28

dict_sol1 = {
'The number of movies in the dataset': d,
'The number of ratings in the dataset': h,
'The number of different genres': i,
'The number of unique users in the dataset': a, 
'The number missing ratings in the reviews dataset': f,
'The average rating given across all ratings': c, 
'The minimum rating given across all ratings': e, 
'The maximum rating given across all ratings': b
}

# Check your solution
t.q1_check(dict_sol1)

Oops!  That doesn't look quite right.  Try again.


In [26]:
print('The number of ratings in the dataset is {}.'.format(reviews.user_id.nunique()))
print('The number of movies in the dataset is {}.'.format(movies.shape[0]))
print('The number of different genres is {}.'.format(len(genres)))
print('The number of unique users in the dataset is {}.'.format(reviews.user_id.nunique()))
print('The number missing ratings in the reviews dataset is {}.'.format(int(reviews.rating.isnull().mean()*reviews.shape[0])))
print('The average rating given across all ratings is {}.'.format(int(np.round(reviews.rating.mean(),0))))
print('The minimum rating given across all ratings is {}.'.format(reviews.rating.min()))
print('The maximum rating given across all ratings is {}.'.format(reviews.rating.max()))

The number of ratings in the dataset is 70191.
The number of movies in the dataset is 36966.
The number of different genres is 28.
The number of unique users in the dataset is 70191.
The number missing ratings in the reviews dataset is 0.
The average rating given across all ratings is 7.
The minimum rating given across all ratings is 0.
The maximum rating given across all ratings is 10.


#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's

#### Reviews
* Create a date out of time stamp
* Create month/day/year columns from timestamp as dummies

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

Movies:

In [29]:
# pull data
create_date = lambda val: val[-5:-1] if val[-1] == ')' else np.nan

# apply the function to pull the data
movies['date'] = movies['movie'].apply(create_date)

# Return century to pull the date
def add_movie_year(val):
    if val[:2] == yr:
        return 1
    else:
        return 0

In [30]:
# apply movies function
for yr in ['18', '19', '20']:
    movies[str(yr) + "00's"] = movies['date'].apply(add_movie_year)

In [31]:
# Function to split and return values for columns
def split_genres(val):
    try:
        if val.find(gene) >-1:
            return 1
        else:
            return 0
    except AttributeError:
        return 0

# Apply function for each genre
for gene in genres:        
    movies[gene] = movies['genre'].apply(split_genres)

In [32]:
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,Thriller,Short,Adult,...,News,Biography,History,Comedy,Fantasy,Western,Action,Musical,Horror,Sci-Fi
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [33]:
import datetime

change_timestamp = lambda val: datetime.datetime.fromtimestamp(int(val)).strftime('%Y-%m-%d %H:%M:%S')

reviews['date'] = reviews['timestamp'].apply(change_timestamp)

In [34]:
reviews_new, movies_new = t.show_clean_dataframes()

FileNotFoundError: [Errno 2] No such file or directory: './reviews_clean.csv'

In [36]:
reviews.to_csv('./reviews_clean.csv')
movies.to_csv('./movies_clean.csv')